In [34]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from geopy.geocoders import Nominatim
import json
import os


def get_current_and_next_7_days_weather(city_name):
	geolocator = Nominatim(user_agent="MyApp")
	location = geolocator.geocode(f"{city_name}, Việt Nam")
	print(location)

	lat = location.latitude
	long = location.longitude

	print("The latitude of the location is: ", lat)
	print("The longitude of the location is: ", long)

	cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
	retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
	openmeteo = openmeteo_requests.Client(session = retry_session)

	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
	url = "https://api.open-meteo.com/v1/forecast"
	params = {
		"latitude": lat,
		"longitude": long,
		"current": ["temperature_2m", "rain"],
		"daily": ["temperature_2m_max", "temperature_2m_min", "uv_index_max", "precipitation_sum"],
		"timezone": "auto"
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Current values. The order of variables needs to be the same as requested.
	current = response.Current()
	current_temperature_2m = current.Variables(0).Value()
	current_rain = current.Variables(1).Value()

	print(f"Current time {current.Time()}")
	print(f"Current temperature_2m {current_temperature_2m}")
	print(f"Current rain {current_rain}")

	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_uv_index_max = daily.Variables(2).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["uv_index_max"] = daily_uv_index_max
	daily_data["precipitation_sum"] = daily_precipitation_sum

	daily_dataframe = pd.DataFrame(data = daily_data)
	display(daily_dataframe)

	return daily_dataframe, current_temperature_2m, current_rain

 
static_asset = r"pfl_app\static\pfl_app\assets"
matched_city_data = os.path.join(static_asset, r"matched_city_data.json")

with open(matched_city_data, "r", encoding="utf-8") as file:
	cities = json.load(file)

for city in cities:
	city_name = city['city']
	daily_dataframe, current_temperature_2m, current_rain = get_current_and_next_7_days_weather(city_name)


Tỉnh Hòa Bình, Việt Nam
The latitude of the location is:  20.6763365
The longitude of the location is:  105.3759952
Coordinates 20.625°N 105.375°E
Elevation 355.0 m asl
Timezone b'Asia/Bangkok' b'+07'
Timezone difference to GMT+0 25200 s
Current time 1731734100
Current temperature_2m 28.75
Current rain 0.0


,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,29.436998,21.587000,2.50,0.0
1,2024-11-16 17:00:00+00:00,28.337000,20.987000,6.15,0.0
2,2024-11-17 17:00:00+00:00,23.186998,20.886999,5.85,4.9
3,2024-11-18 17:00:00+00:00,23.936998,18.636999,3.70,0.0
4,2024-11-19 17:00:00+00:00,23.186998,17.036999,6.25,0.0
5,2024-11-20 17:00:00+00:00,21.386999,16.587000,2.90,0.0
6,2024-11-21 17:00:00+00:00,22.737000,14.837000,1.55,0.0


Tỉnh Sơn La, Việt Nam
The latitude of the location is:  21.2276769
The longitude of the location is:  104.1575944
Coordinates 21.25°N 104.125°E
Elevation 751.0 m asl
Timezone b'Asia/Bangkok' b'+07'
Timezone difference to GMT+0 25200 s
Current time 1731734100
Current temperature_2m 27.200000762939453
Current rain 0.0


,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,27.748501,15.3485,6.20,0.0
1,2024-11-16 17:00:00+00:00,27.148500,16.6985,6.10,0.0
2,2024-11-17 17:00:00+00:00,26.398500,16.2985,5.95,0.0
3,2024-11-18 17:00:00+00:00,23.948500,16.6485,5.60,0.0
4,2024-11-19 17:00:00+00:00,22.698500,14.1485,6.05,0.0
5,2024-11-20 17:00:00+00:00,21.498501,12.3485,5.75,0.0
6,2024-11-21 17:00:00+00:00,21.998501,11.5485,5.70,0.0


Tỉnh Quảng Bình, Việt Nam
The latitude of the location is:  17.509599
The longitude of the location is:  106.4004452
Coordinates 17.375°N 106.25°E
Elevation 978.0 m asl
Timezone b'Asia/Bangkok' b'+07'
Timezone difference to GMT+0 25200 s
Current time 1731734100
Current temperature_2m 25.0
Current rain 0.0


,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,25.095499,16.695499,6.85,0.0
1,2024-11-16 17:00:00+00:00,23.195499,17.395500,6.90,0.0
2,2024-11-17 17:00:00+00:00,19.845499,18.045500,5.85,4.9
3,2024-11-18 17:00:00+00:00,19.195499,15.845499,1.80,4.4
4,2024-11-19 17:00:00+00:00,19.095499,15.895500,0.45,0.0
5,2024-11-20 17:00:00+00:00,17.345499,15.495499,0.30,0.0
6,2024-11-21 17:00:00+00:00,17.645500,15.495499,0.55,0.0


Tỉnh Quảng Nam, Việt Nam
The latitude of the location is:  15.5761698
The longitude of the location is:  108.0527132
Coordinates 15.75°N 108.125°E
Elevation 39.0 m asl
Timezone b'Asia/Ho_Chi_Minh' b'+07'
Timezone difference to GMT+0 25200 s
Current time 1731734100
Current temperature_2m 30.850000381469727
Current rain 0.0


,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,31.129501,23.379501,6.95,0.000000
1,2024-11-16 17:00:00+00:00,29.629501,21.629501,7.15,0.200000
2,2024-11-17 17:00:00+00:00,25.979500,22.879501,7.05,12.999999
3,2024-11-18 17:00:00+00:00,24.579498,22.329498,2.20,9.099999
4,2024-11-19 17:00:00+00:00,27.279499,22.179501,3.90,0.100000
5,2024-11-20 17:00:00+00:00,24.329498,23.429501,0.25,11.799999
6,2024-11-21 17:00:00+00:00,24.979500,23.679501,0.25,21.600000


Tỉnh Lào Cai, Việt Nam
The latitude of the location is:  22.3069302
The longitude of the location is:  104.1829592
Coordinates 22.375°N 104.125°E
Elevation 99.0 m asl
Timezone b'Asia/Bangkok' b'+07'
Timezone difference to GMT+0 25200 s
Current time 1731734100
Current temperature_2m 29.5
Current rain 0.0


,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,31.016001,19.466,5.90,0.0
1,2024-11-16 17:00:00+00:00,30.366001,19.666,5.85,0.0
2,2024-11-17 17:00:00+00:00,28.866001,19.666,5.65,0.0
3,2024-11-18 17:00:00+00:00,25.916000,20.666,2.45,0.0
4,2024-11-19 17:00:00+00:00,25.966000,17.566,5.70,0.0
5,2024-11-20 17:00:00+00:00,26.266001,16.816,5.45,0.0
6,2024-11-21 17:00:00+00:00,25.916000,15.916,3.00,0.0


In [37]:
daily_dataframe

,date,temperature_2m_max,temperature_2m_min,uv_index_max,precipitation_sum
0,2024-11-15 17:00:00+00:00,31.016001,19.466,5.90,0.0
1,2024-11-16 17:00:00+00:00,30.366001,19.666,5.85,0.0
2,2024-11-17 17:00:00+00:00,28.866001,19.666,5.65,0.0
3,2024-11-18 17:00:00+00:00,25.916000,20.666,2.45,0.0
4,2024-11-19 17:00:00+00:00,25.966000,17.566,5.70,0.0
5,2024-11-20 17:00:00+00:00,26.266001,16.816,5.45,0.0
6,2024-11-21 17:00:00+00:00,25.916000,15.916,3.00,0.0
